In [1]:
'''
DCGAN on MNIST using Keras
Author: Rowel Atienza
Project: https://github.com/roatienza/Deep-Learning-Experiments
Dependencies: tensorflow 1.0 and keras 2.0
Usage: python3 dcgan_mnist.py
'''

import numpy as np
import time
from tensorflow.examples.tutorials.mnist import input_data

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# 計算需要的時間
class ElapsedTimer(object):
    def __init__(self):
        self.start_time = time.time()
    def elapsed(self,sec):
        if sec < 60:
            return str(sec) + " sec"
        elif sec < (60 * 60):
            return str(sec / 60) + " min"
        else:
            return str(sec / (60 * 60)) + " hr"
    def elapsed_time(self):
        print("Elapsed: %s " % self.elapsed(time.time() - self.start_time) )
# 宣告類別
class DCGAN(object):
    def __init__(self, img_rows=28, img_cols=28, channel=1):
    #初始化過程宣告影像大小與單色或是彩色
        self.img_rows = img_rows
        self.img_cols = img_cols
        self.channel = channel
        self.D = None   # discriminator
        self.G = None   # generator
        self.AM = None  # adversarial model
        self.DM = None  # discriminator model

    # (W−F+2P)/S+1
    def discriminator(self):
        if self.D:
            return self.D
        self.D = Sequential()
        depth = 64
        dropout = 0.4
        # In: 28 x 28 x 1, depth = 1
        # Out: 14 x 14 x 1, depth=64
        input_shape = (self.img_rows, self.img_cols, self.channel)
        self.D.add(Conv2D(depth*1, 5, strides=2, input_shape=input_shape,\
            padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*2, 5, strides=2, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*4, 5, strides=2, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*8, 5, strides=1, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        # Out: 1-dim probability
        self.D.add(Flatten())
        self.D.add(Dense(1))
        self.D.add(Activation('sigmoid'))
        self.D.summary()
        return self.D

    def generator(self):
        if self.G:
            return self.G
        self.G = Sequential()
        dropout = 0.4
        depth = 64+64+64+64
        dim = 7
        # In: 100
        # Out: dim x dim x depth
        self.G.add(Dense(dim*dim*depth, input_dim=100))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))
        self.G.add(Reshape((dim, dim, depth)))
        self.G.add(Dropout(dropout))

        # In: dim x dim x depth
        # Out: 2*dim x 2*dim x depth/2
        self.G.add(UpSampling2D())
        self.G.add(Conv2DTranspose(int(depth/2), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        self.G.add(UpSampling2D())
        self.G.add(Conv2DTranspose(int(depth/4), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        self.G.add(Conv2DTranspose(int(depth/8), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        # Out: 28 x 28 x 1 grayscale image [0.0,1.0] per pix
        self.G.add(Conv2DTranspose(1, 5, padding='same'))
        self.G.add(Activation('sigmoid'))
        self.G.summary()
        return self.G

    def discriminator_model(self):
        if self.DM:
            return self.DM
        optimizer = RMSprop(lr=0.0002, decay=6e-8)
        self.DM = Sequential()
        self.DM.add(self.discriminator())
        self.DM.compile(loss='binary_crossentropy', optimizer=optimizer,\
            metrics=['accuracy'])
        return self.DM

    def adversarial_model(self):
        if self.AM:
            return self.AM
        optimizer = RMSprop(lr=0.0001, decay=3e-8)
        self.AM = Sequential()
        self.AM.add(self.generator())
        self.AM.add(self.discriminator())
        self.AM.compile(loss='binary_crossentropy', optimizer=optimizer,\
            metrics=['accuracy'])
        return self.AM

class MNIST_DCGAN(object):
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channel = 1

        self.x_train = input_data.read_data_sets("mnist",one_hot=True).train.images
        self.x_train = self.x_train.reshape(-1, self.img_rows, self.img_cols, 1).astype(np.float32)

        self.DCGAN = DCGAN()
        self.discriminator =  self.DCGAN.discriminator_model()
        self.adversarial = self.DCGAN.adversarial_model()
        self.generator = self.DCGAN.generator()

    def train(self, train_steps=2000, batch_size=256, save_interval=0):
        noise_input = None
        if save_interval>0:
            noise_input = np.random.uniform(-1.0, 1.0, size=[16, 100])
        for i in range(train_steps):
            images_train = self.x_train[np.random.randint(0,
                self.x_train.shape[0], size=batch_size), :, :, :]
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
            images_fake = self.generator.predict(noise)
            x = np.concatenate((images_train, images_fake))
            y = np.ones([2*batch_size, 1])
            y[batch_size:, :] = 0
            d_loss = self.discriminator.train_on_batch(x, y)

            y = np.ones([batch_size, 1])
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
            a_loss = self.adversarial.train_on_batch(noise, y)
            log_mesg = "%d: [D loss: %f, acc: %f]" % (i, d_loss[0], d_loss[1])
            log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
            print(log_mesg)
            if save_interval>0:
                if (i+1)%save_interval==0:
                    self.plot_images(save2file=True, samples=noise_input.shape[0],\
                        noise=noise_input, step=(i+1))

    def plot_images(self, save2file=False, fake=True, samples=16, noise=None, step=0):
        filename = 'mnist.png'
        if fake:
            if noise is None:
                noise = np.random.uniform(-1.0, 1.0, size=[samples, 100])
            else:
                filename = "mnist_%d.png" % step
            images = self.generator.predict(noise)
        else:
            i = np.random.randint(0, self.x_train.shape[0], samples)
            images = self.x_train[i, :, :, :]

        plt.figure(figsize=(10,10))
        for i in range(images.shape[0]):
            plt.subplot(4, 4, i+1)
            image = images[i, :, :, :]
            image = np.reshape(image, [self.img_rows, self.img_cols])
            plt.imshow(image, cmap='gray')
            plt.axis('off')
        plt.tight_layout()
        if save2file:
            plt.savefig(filename)
            plt.close('all')
        else:
            plt.show()

In [3]:
# 主要程式碼
mnist_dcgan = MNIST_DCGAN()

timer = ElapsedTimer()
mnist_dcgan.train(train_steps=10000, batch_size=256, save_interval=500)
timer.elapsed_time()


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting mnist\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting mnist\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models

7: [D loss: 0.539374, acc: 0.500000]  [A loss: 0.809158, acc: 0.000000]
8: [D loss: 0.544068, acc: 0.500000]  [A loss: 0.840077, acc: 0.000000]
9: [D loss: 0.522125, acc: 0.501953]  [A loss: 0.835050, acc: 0.000000]
10: [D loss: 0.516806, acc: 0.500000]  [A loss: 0.859171, acc: 0.000000]
11: [D loss: 0.491945, acc: 0.521484]  [A loss: 0.855179, acc: 0.003906]
12: [D loss: 0.503694, acc: 0.501953]  [A loss: 0.903039, acc: 0.000000]
13: [D loss: 0.474135, acc: 0.710938]  [A loss: 0.798022, acc: 0.023438]
14: [D loss: 0.627192, acc: 0.500000]  [A loss: 1.012218, acc: 0.000000]
15: [D loss: 0.569039, acc: 0.929688]  [A loss: 0.781205, acc: 0.062500]
16: [D loss: 0.491387, acc: 0.521484]  [A loss: 0.820975, acc: 0.031250]
17: [D loss: 0.502487, acc: 0.537109]  [A loss: 0.872115, acc: 0.000000]
18: [D loss: 0.469949, acc: 0.660156]  [A loss: 0.858738, acc: 0.007812]
19: [D loss: 0.534849, acc: 0.531250]  [A loss: 0.981287, acc: 0.000000]
20: [D loss: 0.486341, acc: 0.933594]  [A loss: 0.7526

120: [D loss: 0.673646, acc: 0.515625]  [A loss: 1.545550, acc: 0.000000]
121: [D loss: 0.561788, acc: 0.777344]  [A loss: 0.760435, acc: 0.324219]
122: [D loss: 0.677950, acc: 0.500000]  [A loss: 1.421743, acc: 0.000000]
123: [D loss: 0.556473, acc: 0.785156]  [A loss: 0.807315, acc: 0.246094]
124: [D loss: 0.693005, acc: 0.509766]  [A loss: 1.507676, acc: 0.000000]
125: [D loss: 0.573441, acc: 0.757812]  [A loss: 0.737043, acc: 0.351562]
126: [D loss: 0.706112, acc: 0.505859]  [A loss: 1.449392, acc: 0.000000]
127: [D loss: 0.567549, acc: 0.777344]  [A loss: 0.726205, acc: 0.394531]
128: [D loss: 0.695553, acc: 0.503906]  [A loss: 1.334827, acc: 0.000000]
129: [D loss: 0.572260, acc: 0.806641]  [A loss: 0.802416, acc: 0.222656]
130: [D loss: 0.680268, acc: 0.503906]  [A loss: 1.359908, acc: 0.003906]
131: [D loss: 0.576321, acc: 0.783203]  [A loss: 0.789655, acc: 0.257812]
132: [D loss: 0.686597, acc: 0.505859]  [A loss: 1.388897, acc: 0.000000]
133: [D loss: 0.567579, acc: 0.822266]

231: [D loss: 0.664572, acc: 0.617188]  [A loss: 0.680809, acc: 0.546875]
232: [D loss: 0.721651, acc: 0.507812]  [A loss: 1.206909, acc: 0.000000]
233: [D loss: 0.654987, acc: 0.652344]  [A loss: 0.720506, acc: 0.437500]
234: [D loss: 0.729823, acc: 0.498047]  [A loss: 1.247407, acc: 0.000000]
235: [D loss: 0.666267, acc: 0.615234]  [A loss: 0.666746, acc: 0.601562]
236: [D loss: 0.725907, acc: 0.501953]  [A loss: 1.198192, acc: 0.000000]
237: [D loss: 0.672149, acc: 0.625000]  [A loss: 0.717115, acc: 0.421875]
238: [D loss: 0.703064, acc: 0.509766]  [A loss: 1.185472, acc: 0.000000]
239: [D loss: 0.657336, acc: 0.628906]  [A loss: 0.717014, acc: 0.414062]
240: [D loss: 0.705395, acc: 0.503906]  [A loss: 1.160539, acc: 0.003906]
241: [D loss: 0.685095, acc: 0.554688]  [A loss: 0.796405, acc: 0.250000]
242: [D loss: 0.705377, acc: 0.515625]  [A loss: 1.252221, acc: 0.000000]
243: [D loss: 0.668431, acc: 0.589844]  [A loss: 0.693160, acc: 0.527344]
244: [D loss: 0.718647, acc: 0.505859]

342: [D loss: 0.696484, acc: 0.521484]  [A loss: 1.088763, acc: 0.015625]
343: [D loss: 0.665304, acc: 0.609375]  [A loss: 0.711532, acc: 0.445312]
344: [D loss: 0.705693, acc: 0.511719]  [A loss: 1.159341, acc: 0.003906]
345: [D loss: 0.676972, acc: 0.560547]  [A loss: 0.659838, acc: 0.640625]
346: [D loss: 0.713106, acc: 0.505859]  [A loss: 1.111435, acc: 0.000000]
347: [D loss: 0.676212, acc: 0.576172]  [A loss: 0.711715, acc: 0.453125]
348: [D loss: 0.688815, acc: 0.539062]  [A loss: 1.044838, acc: 0.003906]
349: [D loss: 0.665086, acc: 0.617188]  [A loss: 0.737545, acc: 0.402344]
350: [D loss: 0.698730, acc: 0.511719]  [A loss: 1.160812, acc: 0.000000]
351: [D loss: 0.668382, acc: 0.591797]  [A loss: 0.654279, acc: 0.640625]
352: [D loss: 0.712686, acc: 0.515625]  [A loss: 1.138970, acc: 0.000000]
353: [D loss: 0.671467, acc: 0.605469]  [A loss: 0.681525, acc: 0.562500]
354: [D loss: 0.698548, acc: 0.521484]  [A loss: 1.036158, acc: 0.011719]
355: [D loss: 0.683785, acc: 0.541016]

...

9478: [D loss: 0.703832, acc: 0.523438]  [A loss: 0.678335, acc: 0.554688]
9479: [D loss: 0.730413, acc: 0.515625]  [A loss: 0.964558, acc: 0.097656]
9480: [D loss: 0.698714, acc: 0.525391]  [A loss: 0.705487, acc: 0.476562]
9481: [D loss: 0.710259, acc: 0.513672]  [A loss: 0.863922, acc: 0.160156]
9482: [D loss: 0.694346, acc: 0.535156]  [A loss: 0.825619, acc: 0.269531]
9483: [D loss: 0.707165, acc: 0.523438]  [A loss: 0.816951, acc: 0.289062]
9484: [D loss: 0.702070, acc: 0.523438]  [A loss: 0.794843, acc: 0.304688]
9485: [D loss: 0.706741, acc: 0.527344]  [A loss: 0.856356, acc: 0.207031]
9486: [D loss: 0.703557, acc: 0.503906]  [A loss: 0.785338, acc: 0.351562]
9487: [D loss: 0.729637, acc: 0.466797]  [A loss: 0.884098, acc: 0.140625]
9488: [D loss: 0.706512, acc: 0.521484]  [A loss: 0.797599, acc: 0.324219]
9489: [D loss: 0.702722, acc: 0.537109]  [A loss: 0.868808, acc: 0.199219]
9490: [D loss: 0.698805, acc: 0.517578]  [A loss: 0.753339, acc: 0.382812]
9491: [D loss: 0.722345, 

9588: [D loss: 0.692391, acc: 0.519531]  [A loss: 0.772259, acc: 0.363281]
9589: [D loss: 0.720075, acc: 0.482422]  [A loss: 0.829363, acc: 0.242188]
9590: [D loss: 0.697382, acc: 0.542969]  [A loss: 0.815114, acc: 0.261719]
9591: [D loss: 0.701072, acc: 0.529297]  [A loss: 0.865638, acc: 0.164062]
9592: [D loss: 0.694772, acc: 0.542969]  [A loss: 0.757191, acc: 0.363281]
9593: [D loss: 0.703670, acc: 0.527344]  [A loss: 0.866475, acc: 0.160156]
9594: [D loss: 0.699299, acc: 0.511719]  [A loss: 0.774495, acc: 0.304688]
9595: [D loss: 0.698814, acc: 0.529297]  [A loss: 0.807104, acc: 0.289062]
9596: [D loss: 0.712692, acc: 0.529297]  [A loss: 0.903435, acc: 0.144531]
9597: [D loss: 0.704081, acc: 0.507812]  [A loss: 0.783995, acc: 0.335938]
9598: [D loss: 0.715788, acc: 0.503906]  [A loss: 0.964039, acc: 0.105469]
9599: [D loss: 0.708087, acc: 0.515625]  [A loss: 0.732365, acc: 0.449219]
9600: [D loss: 0.735137, acc: 0.490234]  [A loss: 0.964533, acc: 0.093750]
9601: [D loss: 0.700576, 

9698: [D loss: 0.711570, acc: 0.484375]  [A loss: 0.825584, acc: 0.269531]
9699: [D loss: 0.703064, acc: 0.503906]  [A loss: 0.733934, acc: 0.417969]
9700: [D loss: 0.719647, acc: 0.509766]  [A loss: 1.014735, acc: 0.058594]
9701: [D loss: 0.710229, acc: 0.498047]  [A loss: 0.646772, acc: 0.628906]
9702: [D loss: 0.735907, acc: 0.490234]  [A loss: 0.924027, acc: 0.117188]
9703: [D loss: 0.707344, acc: 0.498047]  [A loss: 0.665635, acc: 0.578125]
9704: [D loss: 0.715405, acc: 0.523438]  [A loss: 0.886475, acc: 0.132812]
9705: [D loss: 0.690329, acc: 0.541016]  [A loss: 0.711349, acc: 0.523438]
9706: [D loss: 0.711236, acc: 0.542969]  [A loss: 0.843239, acc: 0.160156]
9707: [D loss: 0.693023, acc: 0.527344]  [A loss: 0.731513, acc: 0.449219]
9708: [D loss: 0.714918, acc: 0.507812]  [A loss: 0.857484, acc: 0.203125]
9709: [D loss: 0.698442, acc: 0.523438]  [A loss: 0.742422, acc: 0.382812]
9710: [D loss: 0.714472, acc: 0.509766]  [A loss: 0.899182, acc: 0.125000]
9711: [D loss: 0.704679, 

9808: [D loss: 0.687866, acc: 0.578125]  [A loss: 0.736632, acc: 0.414062]
9809: [D loss: 0.713199, acc: 0.521484]  [A loss: 0.929113, acc: 0.128906]
9810: [D loss: 0.686180, acc: 0.572266]  [A loss: 0.672367, acc: 0.589844]
9811: [D loss: 0.735723, acc: 0.498047]  [A loss: 0.958844, acc: 0.054688]
9812: [D loss: 0.698811, acc: 0.503906]  [A loss: 0.699950, acc: 0.527344]
9813: [D loss: 0.724060, acc: 0.494141]  [A loss: 0.928008, acc: 0.125000]
9814: [D loss: 0.703828, acc: 0.501953]  [A loss: 0.750399, acc: 0.367188]
9815: [D loss: 0.705259, acc: 0.523438]  [A loss: 0.873584, acc: 0.171875]
9816: [D loss: 0.695723, acc: 0.527344]  [A loss: 0.755004, acc: 0.378906]
9817: [D loss: 0.708787, acc: 0.503906]  [A loss: 0.849562, acc: 0.191406]
9818: [D loss: 0.706337, acc: 0.496094]  [A loss: 0.772162, acc: 0.324219]
9819: [D loss: 0.702797, acc: 0.525391]  [A loss: 0.893678, acc: 0.136719]
9820: [D loss: 0.704275, acc: 0.511719]  [A loss: 0.717835, acc: 0.507812]
9821: [D loss: 0.710423, 

9918: [D loss: 0.718454, acc: 0.515625]  [A loss: 1.005082, acc: 0.042969]
9919: [D loss: 0.702408, acc: 0.531250]  [A loss: 0.689625, acc: 0.519531]
9920: [D loss: 0.729467, acc: 0.464844]  [A loss: 0.900229, acc: 0.093750]
9921: [D loss: 0.721143, acc: 0.460938]  [A loss: 0.729045, acc: 0.472656]
9922: [D loss: 0.718802, acc: 0.503906]  [A loss: 0.862354, acc: 0.140625]
9923: [D loss: 0.702136, acc: 0.513672]  [A loss: 0.754467, acc: 0.390625]
9924: [D loss: 0.702617, acc: 0.509766]  [A loss: 0.837582, acc: 0.203125]
9925: [D loss: 0.692836, acc: 0.546875]  [A loss: 0.800062, acc: 0.269531]
9926: [D loss: 0.704788, acc: 0.501953]  [A loss: 0.791831, acc: 0.300781]
9927: [D loss: 0.694319, acc: 0.550781]  [A loss: 0.831795, acc: 0.199219]
9928: [D loss: 0.692831, acc: 0.542969]  [A loss: 0.790785, acc: 0.320312]
9929: [D loss: 0.703318, acc: 0.492188]  [A loss: 0.795761, acc: 0.261719]
9930: [D loss: 0.698370, acc: 0.531250]  [A loss: 0.808543, acc: 0.261719]
9931: [D loss: 0.702026, 

In [ ]:
mnist_dcgan.plot_images(fake=True)
mnist_dcgan.plot_images(fake=False, save2file=True)